# HW3: 基于Dueling DDQN的配对交易策略

**作者**: 学生姓名  
**日期**: 2025-11-15  

---

## 作业要求

1. 使用LLM建议10个DDQN模型用于配对交易
2. 选择一个模型并生成代码
3. 分割数据为训练集和测试集，并训练模型
4. 使用训练好的模型在测试集上生成预测
5. 评估交易结果

---

# Part 1: 10个DDQN模型建议

## 模型列表

### 1. 基础DDQN (Baseline DDQN)
- **架构**: 3层全连接网络 (128-64-32)
- **优点**: 实现简单，易于调试
- **缺点**: 特征提取能力有限

### 2. **Dueling DDQN** ⭐ (最终选择)
- **架构**: 分离价值流和优势流
- **优点**: 更准确地估计状态价值，训练稳定
- **缺点**: 网络复杂度稍高
- **选择理由**: 性能与复杂度的最佳平衡

### 3. Attention-based DDQN
- **架构**: 多头自注意力机制
- **优点**: 捕捉长期依赖
- **缺点**: 训练时间长

### 4. CNN-DDQN
- **架构**: 1D卷积层提取局部特征
- **优点**: 提取价格走势模式
- **缺点**: 可能丢失全局信息

### 5. LSTM-DDQN
- **架构**: 双层LSTM + 全连接
- **优点**: 擅长处理时间序列
- **缺点**: 训练速度慢

### 6. Hierarchical DDQN
- **架构**: 两层策略网络
- **优点**: 分解复杂决策
- **缺点**: 训练复杂度高

### 7. Ensemble DDQN
- **架构**: 3-5个独立网络集成
- **优点**: 降低方差，提高稳定性
- **缺点**: 计算成本高

### 8. Noisy DDQN
- **架构**: 参数化噪声层
- **优点**: 探索更高效
- **缺点**: 实现稍复杂

### 9. Prioritized Experience Replay DDQN
- **架构**: 基于TD误差的优先级采样
- **优点**: 更高效利用经验
- **缺点**: 内存开销大

### 10. Rainbow DQN
- **架构**: 整合多种改进技术
- **优点**: 性能最优
- **缺点**: 实现难度最高

---

## 最终选择: Dueling DDQN

**理由:**
1. 通过分离状态价值V(s)和动作优势A(s,a)，更准确评估每个动作的价值
2. 在配对交易的买入/卖出/持有决策中表现优异
3. 实现复杂度适中，训练稳定
4. 易于扩展和改进

**核心公式:**
```
Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))
```

---

# Part 2: 环境配置

## 训练模式选择

- **快速模式**: 50 episodes，适合快速验证和演示（约5-10分钟）
- **完整模式**: 500 episodes，完整训练以获得最佳性能（约30-60分钟）

In [ ]:
# ========== 配置参数 ==========
QUICK_MODE = True  # True=快速模式(50 episodes), False=完整模式(500 episodes)

# 根据模式设置参数
if QUICK_MODE:
    N_EPISODES = 50
    N_SAMPLES = 500
    print("⚡ 快速模式: 50 episodes, 500 数据样本")
else:
    N_EPISODES = 500
    N_SAMPLES = 1000
    print("🔥 完整模式: 500 episodes, 1000 数据样本")

# 其他超参数
LEARNING_RATE = 1e-3
GAMMA = 0.99
BATCH_SIZE = 64
BUFFER_SIZE = 10000
TEST_SIZE = 0.2

⚡ 快速模式: 50 episodes, 500 数据样本


: 

## 安装依赖

In [ ]:
# 导入基础库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# 检测设备
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用设备: {device}")
print("✓ 所有库导入成功!")

---

# Part 3: 核心代码实现

## 3.1 Dueling DDQN模型

In [ ]:
class DuelingDQN(nn.Module):
    """Dueling DQN网络架构"""
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super(DuelingDQN, self).__init__()
        
        # 共享特征提取层
        self.feature = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU()
        )
        
        # 状态价值流 V(s)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 1)
        )
        
        # 动作优势流 A(s, a)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, action_dim)
        )
    
    def forward(self, state):
        """Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))"""
        features = self.feature(state)
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)
        
        # 组合价值和优势
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

print("✓ Dueling DQN模型定义完成")

In [ ]:
class ReplayBuffer:
    """经验回放缓冲区"""
    def __init__(self, capacity=10000):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[idx] for idx in indices]
        
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards),
            np.array(next_states),
            np.array(dones)
        )
    
    def __len__(self):
        return len(self.buffer)

print("✓ 经验回放缓冲区定义完成")

In [ ]:
class DuelingDDQNAgent:
    """Dueling DDQN智能体"""
    def __init__(self, state_dim, action_dim, learning_rate=1e-3, gamma=0.99, 
                 epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995,
                 buffer_capacity=10000, batch_size=64, target_update_freq=10, device='cpu'):
        
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.device = device
        
        # 创建主网络和目标网络
        self.policy_net = DuelingDQN(state_dim, action_dim).to(device)
        self.target_net = DuelingDQN(state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        # 优化器
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        
        # 经验回放
        self.replay_buffer = ReplayBuffer(buffer_capacity)
        
        # 训练计数
        self.train_step = 0
    
    def select_action(self, state, training=True):
        """选择动作 (epsilon-greedy)"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.action_dim)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax(dim=1).item()
    
    def store_transition(self, state, action, reward, next_state, done):
        self.replay_buffer.push(state, action, reward, next_state, done)
    
    def update(self):
        """更新网络"""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # 采样批次
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        
        # 转换为张量
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        # 计算当前Q值
        current_q_values = self.policy_net(states).gather(1, actions.unsqueeze(1))
        
        # 计算目标Q值 (Double DQN)
        with torch.no_grad():
            next_actions = self.policy_net(next_states).argmax(dim=1, keepdim=True)
            next_q_values = self.target_net(next_states).gather(1, next_actions)
            target_q_values = rewards.unsqueeze(1) + (1 - dones.unsqueeze(1)) * self.gamma * next_q_values
        
        # 计算损失
        loss = F.mse_loss(current_q_values, target_q_values)
        
        # 优化
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        # 更新epsilon
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)
        
        # 定期更新目标网络
        self.train_step += 1
        if self.train_step % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        
        return loss.item()

print("✓ Dueling DDQN智能体定义完成")

## 3.2 配对交易环境

In [ ]:
class PairTradingEnv:
    """配对交易环境"""
    def __init__(self, data, initial_balance=10000.0, transaction_cost=0.001, 
                 max_position=1, window_size=20):
        self.data = data.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.max_position = max_position
        self.window_size = window_size
        
        # 计算特征
        self._calculate_features()
        
        # 动作空间: 0=平仓, 1=做多, 2=做空, 3=持有
        self.action_space_n = 4
        self.state_dim = self._get_state(self.window_size).shape[0]
        
        self.reset()
    
    def _calculate_features(self):
        """计算技术特征"""
        self.data['spread'] = self.data['price_A'] - self.data['price_B']
        self.data['spread_ma'] = self.data['spread'].rolling(window=self.window_size).mean()
        self.data['spread_std'] = self.data['spread'].rolling(window=self.window_size).std()
        self.data['z_score'] = (self.data['spread'] - self.data['spread_ma']) / (self.data['spread_std'] + 1e-8)
        self.data['spread_return'] = self.data['spread'].pct_change()
        self.data = self.data.fillna(0)
    
    def reset(self):
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.position = 0
        self.entry_price = 0.0
        self.total_trades = 0
        self.winning_trades = 0
        self.equity_curve = [self.initial_balance]
        return self._get_state(self.current_step)
    
    def _get_state(self, step):
        """获取状态"""
        if step < self.window_size:
            step = self.window_size
        
        current_spread = self.data.loc[step, 'spread']
        z_score = self.data.loc[step, 'z_score']
        spread_ma = self.data.loc[step, 'spread_ma']
        spread_std = self.data.loc[step, 'spread_std']
        
        spread_history = self.data.loc[max(0, step-5):step, 'spread'].values
        if len(spread_history) < 6:
            spread_history = np.pad(spread_history, (6-len(spread_history), 0), 'constant')
        
        if spread_std > 0:
            spread_history_norm = (spread_history - spread_ma) / spread_std
        else:
            spread_history_norm = spread_history
        
        position_feature = self.position / self.max_position
        
        if self.position != 0:
            unrealized_pnl = (current_spread - self.entry_price) * self.position
            unrealized_pnl_pct = unrealized_pnl / self.balance
        else:
            unrealized_pnl_pct = 0.0
        
        state = np.array([
            z_score,
            spread_std,
            position_feature,
            unrealized_pnl_pct,
            *spread_history_norm
        ], dtype=np.float32)
        
        return state
    
    def step(self, action):
        """执行动作"""
        current_spread = self.data.loc[self.current_step, 'spread']
        reward = 0.0
        trade_executed = False
        
        # 动作执行逻辑
        if action == 0:  # 平仓
            if self.position != 0:
                pnl = (current_spread - self.entry_price) * self.position
                cost = abs(self.position) * self.transaction_cost * self.balance
                net_pnl = pnl - cost
                self.balance += net_pnl
                reward = net_pnl / self.initial_balance
                self.total_trades += 1
                if net_pnl > 0:
                    self.winning_trades += 1
                self.position = 0
                self.entry_price = 0.0
                trade_executed = True
        
        elif action == 1:  # 做多
            if self.position <= 0:
                if self.position < 0:
                    pnl = (current_spread - self.entry_price) * self.position
                    cost = abs(self.position) * self.transaction_cost * self.balance
                    self.balance += pnl - cost
                self.position = self.max_position
                self.entry_price = current_spread
                cost = self.transaction_cost * self.balance
                self.balance -= cost
                trade_executed = True
        
        elif action == 2:  # 做空
            if self.position >= 0:
                if self.position > 0:
                    pnl = (current_spread - self.entry_price) * self.position
                    cost = abs(self.position) * self.transaction_cost * self.balance
                    self.balance += pnl - cost
                self.position = -self.max_position
                self.entry_price = current_spread
                cost = self.transaction_cost * self.balance
                self.balance -= cost
                trade_executed = True
        
        # 持仓浮动盈亏
        if self.position != 0 and not trade_executed:
            unrealized_pnl = (current_spread - self.entry_price) * self.position
            reward = unrealized_pnl / self.initial_balance * 0.1
        
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        
        # 强制平仓
        if done and self.position != 0:
            final_spread = self.data.loc[self.current_step, 'spread']
            pnl = (final_spread - self.entry_price) * self.position
            cost = abs(self.position) * self.transaction_cost * self.balance
            self.balance += pnl - cost
            self.position = 0
        
        self.equity_curve.append(self.balance)
        next_state = self._get_state(self.current_step) if not done else np.zeros(self.state_dim)
        
        info = {
            'balance': self.balance,
            'position': self.position,
            'total_trades': self.total_trades,
            'winning_trades': self.winning_trades,
            'trade_executed': trade_executed
        }
        
        return next_state, reward, done, info
    
    def get_performance_metrics(self):
        """计算性能指标"""
        equity_curve = np.array(self.equity_curve)
        returns = np.diff(equity_curve) / equity_curve[:-1]
        
        total_return = (self.balance - self.initial_balance) / self.initial_balance
        sharpe_ratio = np.mean(returns) / (np.std(returns) + 1e-8) * np.sqrt(252)
        
        cummax = np.maximum.accumulate(equity_curve)
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown = np.min(drawdown)
        
        win_rate = self.winning_trades / self.total_trades if self.total_trades > 0 else 0
        
        return {
            'total_return': total_return,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'total_trades': self.total_trades,
            'final_balance': self.balance
        }

print("✓ 配对交易环境定义完成")

## 3.3 数据生成和处理

In [ ]:
def generate_synthetic_pair_data(n_samples=1000, mean_reversion_speed=0.1, 
                                volatility=0.02, random_seed=42):
    """生成合成的配对交易数据"""
    np.random.seed(random_seed)
    
    base_price = 100.0
    price_a = np.zeros(n_samples)
    price_b = np.zeros(n_samples)
    
    price_a[0] = base_price
    price_b[0] = base_price
    
    spread_mean = 0.0
    spread = np.zeros(n_samples)
    
    for t in range(1, n_samples):
        spread[t] = (spread[t-1] * (1 - mean_reversion_speed) + 
                    spread_mean * mean_reversion_speed + 
                    np.random.normal(0, volatility))
        
        common_shock = np.random.normal(0, volatility)
        price_a[t] = price_a[t-1] * (1 + common_shock + np.random.normal(0, volatility * 0.5))
        price_b[t] = price_a[t] - spread[t]
    
    df = pd.DataFrame({
        'price_A': price_a,
        'price_B': price_b
    })
    
    return df


def split_train_test(data, test_size=0.2):
    """时间序列顺序分割"""
    split_idx = int(len(data) * (1 - test_size))
    train_data = data.iloc[:split_idx].reset_index(drop=True)
    test_data = data.iloc[split_idx:].reset_index(drop=True)
    return train_data, test_data

print("✓ 数据处理函数定义完成")

---

# Part 4: 数据准备（任务3 - 数据分割）

In [ ]:
# 生成数据
print(f"生成 {N_SAMPLES} 个交易日的配对数据...")
data = generate_synthetic_pair_data(n_samples=N_SAMPLES)

# 可视化
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(data['price_A'], label='Asset A', linewidth=2)
axes[0].plot(data['price_B'], label='Asset B', linewidth=2)
axes[0].set_title('资产价格序列', fontsize=14, fontweight='bold')
axes[0].set_ylabel('价格')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

spread = data['price_A'] - data['price_B']
axes[1].plot(spread, color='purple', linewidth=2)
axes[1].axhline(y=spread.mean(), color='r', linestyle='--', label='均值')
axes[1].fill_between(range(len(spread)), 
                      spread.mean() - spread.std(), 
                      spread.mean() + spread.std(), 
                      alpha=0.3, color='red', label='±1σ')
axes[1].set_title('价差序列（均值回归特性）', fontsize=14, fontweight='bold')
axes[1].set_xlabel('时间')
axes[1].set_ylabel('价差')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✓ 数据生成完成: {len(data)} 个交易日")

In [ ]:
# 分割训练集和测试集
train_data, test_data = split_train_test(data, test_size=TEST_SIZE)

print(f"训练集大小: {len(train_data)} 个交易日 ({(1-TEST_SIZE)*100:.0f}%)")
print(f"测试集大小: {len(test_data)} 个交易日 ({TEST_SIZE*100:.0f}%)")
print("✓ 数据分割完成")

---

# Part 5: 模型训练（任务3 - 训练模型）

In [ ]:
# 创建环境获取维度
temp_env = PairTradingEnv(train_data)
state_dim = temp_env.state_dim
action_dim = temp_env.action_space_n

print(f"状态维度: {state_dim}")
print(f"动作维度: {action_dim}")

# 创建智能体
agent = DuelingDDQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    learning_rate=LEARNING_RATE,
    gamma=GAMMA,
    epsilon_start=1.0,
    epsilon_end=0.01,
    epsilon_decay=0.995,
    buffer_capacity=BUFFER_SIZE,
    batch_size=BATCH_SIZE,
    target_update_freq=10,
    device=device
)

print("✓ 智能体创建完成")
print(f"设备: {device}")

In [ ]:
# 训练
print(f"\n开始训练 ({N_EPISODES} episodes)...\n")

episode_rewards = []
performance_metrics = []
losses = []
best_reward = -np.inf

for episode in tqdm(range(N_EPISODES), desc="训练进度"):
    env = PairTradingEnv(train_data)
    state = env.reset()
    
    episode_reward = 0
    episode_loss = []
    done = False
    
    while not done:
        action = agent.select_action(state, training=True)
        next_state, reward, done, info = env.step(action)
        agent.store_transition(state, action, reward, next_state, done)
        
        loss = agent.update()
        if loss is not None:
            episode_loss.append(loss)
        
        episode_reward += reward
        state = next_state
    
    episode_rewards.append(episode_reward)
    losses.append(np.mean(episode_loss) if episode_loss else 0)
    metrics = env.get_performance_metrics()
    performance_metrics.append(metrics)
    
    # 定期打印
    if (episode + 1) % max(10, N_EPISODES // 10) == 0:
        recent_rewards = episode_rewards[-10:]
        recent_returns = [m['total_return'] for m in performance_metrics[-10:]]
        recent_sharpe = [m['sharpe_ratio'] for m in performance_metrics[-10:]]
        
        print(f"\nEpisode {episode + 1}/{N_EPISODES}")
        print(f"  平均奖励: {np.mean(recent_rewards):.4f}")
        print(f"  平均收益率: {np.mean(recent_returns)*100:.2f}%")
        print(f"  平均夏普比率: {np.mean(recent_sharpe):.4f}")
        print(f"  Epsilon: {agent.epsilon:.4f}")
    
    # 保存最佳模型
    if episode_reward > best_reward:
        best_reward = episode_reward
        best_episode = episode + 1

print(f"\n✓ 训练完成!")
print(f"最佳episode: {best_episode}, 奖励: {best_reward:.4f}")

In [ ]:
# 可视化训练过程
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Episode奖励
axes[0, 0].plot(episode_rewards, alpha=0.6, label='Episode奖励')
axes[0, 0].plot(pd.Series(episode_rewards).rolling(10).mean(), label='10期移动平均', linewidth=2)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('总奖励')
axes[0, 0].set_title('训练奖励曲线')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 收益率
returns = [m['total_return'] for m in performance_metrics]
axes[0, 1].plot(returns, alpha=0.6, label='总收益率')
axes[0, 1].plot(pd.Series(returns).rolling(10).mean(), label='10期移动平均', linewidth=2)
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('收益率')
axes[0, 1].set_title('总收益率曲线')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# 夏普比率
sharpe_ratios = [m['sharpe_ratio'] for m in performance_metrics]
axes[1, 0].plot(sharpe_ratios, alpha=0.6, label='夏普比率')
axes[1, 0].plot(pd.Series(sharpe_ratios).rolling(10).mean(), label='10期移动平均', linewidth=2)
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('夏普比率')
axes[1, 0].set_title('夏普比率曲线')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# 损失
axes[1, 1].plot(losses, alpha=0.6, label='训练损失')
axes[1, 1].plot(pd.Series(losses).rolling(10).mean(), label='10期移动平均', linewidth=2)
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('损失')
axes[1, 1].set_title('训练损失曲线')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

# Part 6: 测试集预测（任务4）

In [ ]:
# 在测试集上运行
print("在测试集上生成预测...")

test_env = PairTradingEnv(test_data)
state = test_env.reset()
done = False
step = 0

trade_history = []

while not done:
    # 使用训练好的策略（不探索）
    action = agent.select_action(state, training=False)
    next_state, reward, done, info = test_env.step(action)
    
    trade_history.append({
        'step': step,
        'action': action,
        'position': info['position'],
        'balance': info['balance'],
        'reward': reward,
        'trade_executed': info['trade_executed']
    })
    
    state = next_state
    step += 1

# 转换为DataFrame
df_history = pd.DataFrame(trade_history)

print(f"✓ 测试完成，共 {len(trade_history)} 个时间步")
print(f"\n前10行交易记录:")
print(df_history.head(10))

---

# Part 7: 交易结果评估（任务5）

In [ ]:
# 计算性能指标
metrics = test_env.get_performance_metrics()

print("="*60)
print("测试集评估结果")
print("="*60)
print(f"总收益率:       {metrics['total_return']*100:.2f}%")
print(f"夏普比率:       {metrics['sharpe_ratio']:.4f}")
print(f"最大回撤:       {metrics['max_drawdown']*100:.2f}%")
print(f"总交易次数:     {metrics['total_trades']}")
print(f"盈利交易次数:   {int(metrics['win_rate'] * metrics['total_trades'])}")
print(f"胜率:           {metrics['win_rate']*100:.2f}%")
print(f"初始资金:       $10,000.00")
print(f"最终资金:       ${metrics['final_balance']:.2f}")
print(f"净收益:         ${metrics['final_balance'] - 10000:.2f}")
print("="*60)

In [ ]:
# 详细评估报告
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. 权益曲线
ax1 = fig.add_subplot(gs[0, :])
equity = np.array(test_env.equity_curve)
ax1.plot(equity, linewidth=2, label='权益曲线', color='steelblue')
ax1.axhline(y=10000, color='r', linestyle='--', alpha=0.5, label='初始资金')
ax1.fill_between(range(len(equity)), 10000, equity, alpha=0.3)
ax1.set_xlabel('时间步', fontsize=12)
ax1.set_ylabel('账户权益 ($)', fontsize=12)
ax1.set_title('测试集权益曲线', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 2. 回撤曲线
ax2 = fig.add_subplot(gs[1, 0])
cummax = np.maximum.accumulate(equity)
drawdown = (equity - cummax) / cummax * 100
ax2.fill_between(range(len(drawdown)), 0, drawdown, color='red', alpha=0.5)
ax2.plot(drawdown, color='darkred', linewidth=1.5)
ax2.set_xlabel('时间步', fontsize=12)
ax2.set_ylabel('回撤 (%)', fontsize=12)
ax2.set_title(f'回撤分析 (最大回撤: {metrics["max_drawdown"]*100:.2f}%)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. 收益分布
ax3 = fig.add_subplot(gs[1, 1])
returns = np.diff(equity) / equity[:-1] * 100
ax3.hist(returns, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
ax3.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax3.axvline(x=np.mean(returns), color='g', linestyle='--', 
           linewidth=2, label=f'均值: {np.mean(returns):.4f}%')
ax3.set_xlabel('收益率 (%)', fontsize=12)
ax3.set_ylabel('频数', fontsize=12)
ax3.set_title('收益率分布', fontsize=13, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3)

# 4. 持仓分析
ax4 = fig.add_subplot(gs[2, 0])
position_counts = df_history['position'].value_counts()
colors = ['gray', 'green', 'red']
labels = ['空仓 (0)', '做多 (1)', '做空 (-1)']
positions_to_show = []
counts_to_show = []
colors_to_show = []
for i, pos in enumerate([0, 1, -1]):
    if pos in position_counts.index:
        positions_to_show.append(labels[i])
        counts_to_show.append(position_counts[pos])
        colors_to_show.append(colors[i])
ax4.bar(range(len(counts_to_show)), counts_to_show, color=colors_to_show, alpha=0.7)
ax4.set_xticks(range(len(positions_to_show)))
ax4.set_xticklabels(positions_to_show)
ax4.set_ylabel('时间步数', fontsize=12)
ax4.set_title('持仓分布', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

# 5. 动作分布
ax5 = fig.add_subplot(gs[2, 1])
action_counts = df_history['action'].value_counts().sort_index()
action_labels = ['平仓', '做多', '做空', '持有']
action_colors = ['gray', 'green', 'red', 'blue']
bars = ax5.bar(range(len(action_counts)), action_counts.values,
              color=[action_colors[i] for i in action_counts.index], alpha=0.7)
ax5.set_xticks(range(len(action_counts)))
ax5.set_xticklabels([action_labels[i] for i in action_counts.index])
ax5.set_ylabel('执行次数', fontsize=12)
ax5.set_title('动作分布', fontsize=13, fontweight='bold')
ax5.grid(True, alpha=0.3, axis='y')

for bar in bars:
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Dueling DDQN配对交易策略 - 测试集评估报告',
            fontsize=16, fontweight='bold', y=0.995)

plt.show()

print("✓ 评估报告生成完成")

---

# Part 8: 总结

## 作业完成情况

✅ **任务1**: 使用LLM建议了10个DDQN模型，详见Part 1  
✅ **任务2**: 选择了Dueling DDQN并实现完整代码  
✅ **任务3**: 分割数据为训练集(80%)和测试集(20%)，并训练模型  
✅ **任务4**: 在测试集上生成交易预测  
✅ **任务5**: 完成交易结果评估，包括多维度指标和可视化报告  

## 技术总结

### 选择的模型: Dueling DDQN
- **核心思想**: 分离状态价值V(s)和动作优势A(s,a)
- **数学公式**: Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))
- **主要优势**: 更准确的Q值估计，训练更稳定

### 实现细节
- **状态空间**: 11维（z-score, 价差统计, 持仓, 浮盈, 历史价差）
- **动作空间**: 4个离散动作（平仓/做多/做空/持有）
- **奖励设计**: 实际盈亏 + 浮动盈亏（小权重）
- **环境特性**: 真实交易成本建模（0.1%）

### 性能表现
根据上述评估结果：
- 总收益率、夏普比率、最大回撤等指标
- 交易频率和胜率
- 策略的稳定性和风险控制能力

## 改进方向
1. 添加优先经验回放(PER)提高样本利用效率
2. 实现动态仓位管理而非固定满仓
3. 整合注意力机制处理时间序列
4. 尝试Rainbow DQN整合多种技术

---

**项目完成！** 🎉